In [3]:
from bson import ObjectId
import numpy as np
from pokeai.util import json_load, pickle_load, ROOT_DIR, DATASET_DIR, yaml_dump
from collections import Counter
import json
import yaml
import os
import copy

In [4]:
trainer_param_tmpl = yaml.safe_load("""
battles: 100000
party_tags: []
tags: []
trainer:
  dqn_params:
    epsilon: 0.3
    epsilon_decay: 2.0e-06
    gamma: 0.95
    lr: 0.0004
  feature_params: {}
  model_params:
    bn: false
    n_channels: 16
    n_layers: 3
""")

In [5]:
tag_prefix = "rl_loop_200815_1_"

In [6]:
file_base_dir = r"D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815"

In [11]:
cmds = ""
last_trainer_id = None
for iternum in range(10):
    party_allonce = f"{tag_prefix}allonce_{iternum}"
    party_prev_gen = f"{tag_prefix}hc_{iternum}"
    if iternum == 0:
        cmds += f"""
python -m pokeai.ai.generate_party {party_prev_gen} -n 871 -r finalgoodmove1vs1
"""
    train_param = copy.deepcopy(trainer_param_tmpl)
    train_param["party_tags"] = [party_allonce, party_prev_gen]
    train_param["tags"] = [f"{tag_prefix}{iternum}"]
    train_param_file = os.path.join(file_base_dir, f"{tag_prefix}{iternum}.yaml")
    yaml_dump(train_param, train_param_file)
    trainer_id = ObjectId()
    party_gen = f"{tag_prefix}hc_{iternum+1}"
    resume_trainer_id = f"--resume_trainer_id {last_trainer_id}" if last_trainer_id else ""
    comment_info = json.dumps({"iternum": iternum, "trainer_id": str(trainer_id), "train_party_tags": train_param["party_tags"], "gen_party_tag": party_gen})
    cmds += f"""
python -m pokeai.ai.generate_party {party_allonce} --all_pokemon_once -r finalgoodmove1vs1
python -m pokeai.ai.generic_move_model.rl_train {train_param_file} --trainer_id {trainer_id} {resume_trainer_id}
python -m pokeai.ai.party_feature.hillclimb_party {trainer_id} {party_gen} -r finalgoodmove1vs1 -n 871 --generations 100 --populations 10 --party_feature_penalty 1.0
rem {comment_info}
"""
    last_trainer_id = trainer_id

print(cmds)
with open(os.path.join(file_base_dir, f"{tag_prefix}run.bat"), "w") as f:
    f.write(cmds)



python -m pokeai.ai.generate_party rl_loop_200815_1_hc_0 -n 871 -r finalgoodmove1vs1

python -m pokeai.ai.generate_party rl_loop_200815_1_allonce_0 --all_pokemon_once -r finalgoodmove1vs1
python -m pokeai.ai.generic_move_model.rl_train D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815\rl_loop_200815_1_0.yaml --trainer_id 5f37a6829949403bf6af940b 
python -m pokeai.ai.party_feature.hillclimb_party 5f37a6829949403bf6af940b rl_loop_200815_1_hc_1 -r finalgoodmove1vs1 -n 871 --generations 100 --populations 10 --party_feature_penalty 1.0
rem {"iternum": 0, "trainer_id": "5f37a6829949403bf6af940b", "train_party_tags": ["rl_loop_200815_1_allonce_0", "rl_loop_200815_1_hc_0"], "gen_party_tag": "rl_loop_200815_1_hc_1"}

python -m pokeai.ai.generate_party rl_loop_200815_1_allonce_1 --all_pokemon_once -r finalgoodmove1vs1
python -m pokeai.ai.generic_move_model.rl_train D:\dev\pokeai\pokeai\experiment\gmm\rl\rl_loop_200815\rl_loop_200815_1_1.yaml --trainer_id 5f37a6829949403bf6af940c --resume_tr